In [1]:
from util import *

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
Connected DB CINS_SMY - conn
Connected oraDW_ANALYTICS - conn_aly


In [2]:
query11 = """WITH 
T0 AS (
    SELECT * FROM CINS_REACTIVATED_CASA_CUST_STT_CHG_01112023),
T1 AS (
    SELECT * FROM CINS_REACTIVATED_CASA_SCORE_01112023),
T2 AS (
    SELECT *
        FROM (SELECT T0.*, T1.SCORE, T1.RANK
        FROM T0 INNER JOIN T1 ON T0.CUSTOMER_CDE = T1.CUSTOMER_CDE
        ORDER BY SCORE DESC)),
T21 AS (
    SELECT * FROM DW_ANALYTICS.DW_CUST_PRODUCT_LOC_FCT
    WHERE SD_TKTT = 1 AND PROCESS_DT = TO_DATE('31-10-2023', 'DD-MM-YY')),
T3 AS (
    SELECT T21.*, T2.SCORE, T2.CUST_STT, T2.CUST_STT_LAG, T2.CUST_STT_CHG, T2.REACTIVATED
    FROM T21 
    INNER JOIN T2 ON T21.CUSTOMER_CDE = T2.CUSTOMER_CDE),
T4 AS (
    SELECT * FROM DW_ANALYTICS.DW_ORG_LOCATION_DIM),
T5 AS (
    SELECT * FROM DW_ANALYTICS.DW_CUSTOMER_FULL_DIM),
T6 AS (
    SELECT * FROM DW_ANALYTICS.DW_DEPOSIT_FCT),
T7 AS (
    SELECT * FROM DW_ANALYTICS.DW_LOAN_FCT),
T8 AS (
    SELECT * FROM DW_ANALYTICS.DW_SBVCODE_DIM),
T9 AS (
    SELECT * FROM DW_ANALYTICS.DW_CUSTOMER_DIM
    WHERE ACTIVE = 1),
T10 AS (
    SELECT * FROM CINS_FEATURE_STORE_REACTIVATED_01112023
    WHERE FTR_NM='CASA_DAY_SINCE_LAST_TXN_CT_36M'),
D1 AS (
    SELECT 
    T4.AREA_NAME, T5.SUB_BRANCH_CDE, T4.SUB_BRANCH_NAME, 
    T5.CUSTOMER_CDE, T5.FULL_NAME, T5.EMAIL, T5.MOBILE_PHONE, 
    T3.CUST_TYPE, T3.CUST_STATUS, T3.CUST_OPEN_DT,
    T3.TKTT, T3.TK, T3.VAY, T3.BH, T3.PAY, T3.IB, T3.MB, T3.SP_KHAC, 
    T3.LASTEST_TRANS_DT, T10.FTR_VAL AS CASA_DAY_SINCE_LAST_TXN,
    T9.DAO_CDE,
    T1.SCORE, T1.RANK, T3.REACTIVATED
    FROM T5
    INNER JOIN T3 ON T5.CUSTOMER_CDE = T3.CUSTOMER_CDE
    INNER JOIN T4 ON T5.SUB_BRANCH_CDE = T4.SUB_BRANCH_CDE
    INNER JOIN T1 ON T5.CUSTOMER_CDE = T1.CUSTOMER_CDE
    INNER JOIN T9 ON T5.CUSTOMER_CDE = T9.CUSTOMER_CDE
    INNER JOIN T10 ON T5.CUSTOMER_CDE = T10.CUSTOMER_CDE),
D2 AS (
    SELECT 
    T1.CUSTOMER_CDE, SUM(ACTUAL_BAL_LCL) AS TOTAL_BAL
    FROM T1 
    LEFT JOIN T6 ON T1.CUSTOMER_CDE = T6.CUSTOMER_CDE
    INNER JOIN T8 ON T8.SBVCODE_LVL_3 = T6.SBVCODE_LVL_3
    WHERE T8.APPLICATION = 'DP' AND T6.PROCESS_DT = TO_DATE('31-10-2023', 'DD-MM-YY')
    GROUP BY T1.CUSTOMER_CDE),
D3 AS (
    SELECT 
    T1.CUSTOMER_CDE, SUM(OS_AMT_LCL) AS TOTAL_LOAN
    FROM T1 
    LEFT JOIN T7 ON T1.CUSTOMER_CDE = T7.CUSTOMER_CDE
    INNER JOIN T8 ON T8.SBVCODE_LVL_3 = T7.SBVCODE_LVL_3
    WHERE T8.APPLICATION = 'LN' AND T7.PROCESS_DT = TO_DATE('31-10-2023', 'DD-MM-YY')
    GROUP BY T1.CUSTOMER_CDE),
D4 AS (
    SELECT 
    D1.AREA_NAME, D1.SUB_BRANCH_CDE, D1.SUB_BRANCH_NAME, 
    D1.CUSTOMER_CDE, D1.FULL_NAME, D1.EMAIL, D1.MOBILE_PHONE, 
    D1.CUST_TYPE, D1.CUST_STATUS, D1.CUST_OPEN_DT,
    D1.TKTT, D1.TK, D1.VAY, D1.BH, D1.PAY, D1.IB, D1.MB, D1.SP_KHAC, 
    NVL(D2.TOTAL_BAL, 0) AS TOTAL_BAL, 
    NVL(D3.TOTAL_LOAN, 0) AS TOTAL_LOAN,
    D1.LASTEST_TRANS_DT, 
    CASE WHEN D1.CASA_DAY_SINCE_LAST_TXN IS NULL THEN NULL ELSE ROUND(D1.CASA_DAY_SINCE_LAST_TXN / 30) END AS MONTH_TYPE,
    D1.DAO_CDE,
    D1.CASA_DAY_SINCE_LAST_TXN,
    D1.SCORE, D1.RANK, D1.REACTIVATED
    FROM D1 
    LEFT JOIN D2 ON D1.CUSTOMER_CDE = D2.CUSTOMER_CDE
    LEFT JOIN D3 ON D1.CUSTOMER_CDE = D3.CUSTOMER_CDE)
SELECT *
FROM D4
"""

In [3]:
query10 = """WITH 
T0 AS (
    SELECT * FROM CINS_REACTIVATED_CASA_CUST_STT_CHG_01082023),
T1 AS (
    SELECT * FROM CINS_REACTIVATED_CASA_SCORE_01082023),
T2 AS (
    SELECT *
        FROM (SELECT T0.*, T1.SCORE, T1.RANK
        FROM T0 INNER JOIN T1 ON T0.CUSTOMER_CDE = T1.CUSTOMER_CDE
        ORDER BY SCORE DESC)),
T21 AS (
    SELECT * FROM DW_ANALYTICS.DW_CUST_PRODUCT_LOC_FCT
    WHERE SD_TKTT = 1 AND PROCESS_DT = TO_DATE('31-07-2023', 'DD-MM-YY')),
T3 AS (
    SELECT T21.*, T2.SCORE, T2.CUST_STT, T2.CUST_STT_LAG, T2.CUST_STT_CHG, T2.REACTIVATED
    FROM T21 
    INNER JOIN T2 ON T21.CUSTOMER_CDE = T2.CUSTOMER_CDE),
T4 AS (
    SELECT * FROM DW_ANALYTICS.DW_ORG_LOCATION_DIM),
T5 AS (
    SELECT * FROM DW_ANALYTICS.DW_CUSTOMER_FULL_DIM),
T6 AS (
    SELECT * FROM DW_ANALYTICS.DW_DEPOSIT_FCT),
T7 AS (
    SELECT * FROM DW_ANALYTICS.DW_LOAN_FCT),
T8 AS (
    SELECT * FROM DW_ANALYTICS.DW_SBVCODE_DIM),
T9 AS (
    SELECT * FROM DW_ANALYTICS.DW_CUSTOMER_DIM
    WHERE ACTIVE = 1),
T10 AS (
    SELECT * FROM CINS_FEATURE_STORE_REACTIVATED_01082023
    WHERE FTR_NM='CASA_DAY_SINCE_LAST_TXN_CT_36M'),
D1 AS (
    SELECT 
    T4.AREA_NAME, T5.SUB_BRANCH_CDE, T4.SUB_BRANCH_NAME, 
    T5.CUSTOMER_CDE, T5.FULL_NAME, T5.EMAIL, T5.MOBILE_PHONE, 
    T3.CUST_TYPE, T3.CUST_STATUS, T3.CUST_OPEN_DT,
    T3.TKTT, T3.TK, T3.VAY, T3.BH, T3.PAY, T3.IB, T3.MB, T3.SP_KHAC, 
    T3.LASTEST_TRANS_DT, T10.FTR_VAL AS CASA_DAY_SINCE_LAST_TXN,
    T9.DAO_CDE,
    T1.SCORE, T1.RANK, T3.REACTIVATED
    FROM T5
    INNER JOIN T3 ON T5.CUSTOMER_CDE = T3.CUSTOMER_CDE
    INNER JOIN T4 ON T5.SUB_BRANCH_CDE = T4.SUB_BRANCH_CDE
    INNER JOIN T1 ON T5.CUSTOMER_CDE = T1.CUSTOMER_CDE
    INNER JOIN T9 ON T5.CUSTOMER_CDE = T9.CUSTOMER_CDE
    INNER JOIN T10 ON T5.CUSTOMER_CDE = T10.CUSTOMER_CDE),
D2 AS (
    SELECT 
    T1.CUSTOMER_CDE, SUM(ACTUAL_BAL_LCL) AS TOTAL_BAL
    FROM T1 
    LEFT JOIN T6 ON T1.CUSTOMER_CDE = T6.CUSTOMER_CDE
    INNER JOIN T8 ON T8.SBVCODE_LVL_3 = T6.SBVCODE_LVL_3
    WHERE T8.APPLICATION = 'DP' AND T6.PROCESS_DT = TO_DATE('31-07-2023', 'DD-MM-YY')
    GROUP BY T1.CUSTOMER_CDE),
D3 AS (
    SELECT 
    T1.CUSTOMER_CDE, SUM(OS_AMT_LCL) AS TOTAL_LOAN
    FROM T1 
    LEFT JOIN T7 ON T1.CUSTOMER_CDE = T7.CUSTOMER_CDE
    INNER JOIN T8 ON T8.SBVCODE_LVL_3 = T7.SBVCODE_LVL_3
    WHERE T8.APPLICATION = 'LN' AND T7.PROCESS_DT = TO_DATE('31-07-2023', 'DD-MM-YY')
    GROUP BY T1.CUSTOMER_CDE),
D4 AS (
    SELECT 
    D1.AREA_NAME, D1.SUB_BRANCH_CDE, D1.SUB_BRANCH_NAME, 
    D1.CUSTOMER_CDE, D1.FULL_NAME, D1.EMAIL, D1.MOBILE_PHONE, 
    D1.CUST_TYPE, D1.CUST_STATUS, D1.CUST_OPEN_DT,
    D1.TKTT, D1.TK, D1.VAY, D1.BH, D1.PAY, D1.IB, D1.MB, D1.SP_KHAC, 
    NVL(D2.TOTAL_BAL, 0) AS TOTAL_BAL, 
    NVL(D3.TOTAL_LOAN, 0) AS TOTAL_LOAN,
    D1.LASTEST_TRANS_DT, 
    CASE WHEN D1.CASA_DAY_SINCE_LAST_TXN IS NULL THEN NULL ELSE ROUND(D1.CASA_DAY_SINCE_LAST_TXN / 30) END AS MONTH_TYPE,
    D1.DAO_CDE,
    D1.CASA_DAY_SINCE_LAST_TXN,
    D1.SCORE, D1.RANK, D1.REACTIVATED
    FROM D1 
    LEFT JOIN D2 ON D1.CUSTOMER_CDE = D2.CUSTOMER_CDE
    LEFT JOIN D3 ON D1.CUSTOMER_CDE = D3.CUSTOMER_CDE)
SELECT *
FROM D4
"""

In [4]:
portfolio_10 = download_or_reload('data/PORTFOLIO_08', query10, update=False)

In [ ]:
portfolio_11 = download_or_reload('data/PORTFOLIO_11', query11, update=False)

In [ ]:
portfolio_10['CASA_DAY_SINCE_LAST_TXN'] = portfolio_10['CASA_DAY_SINCE_LAST_TXN'].astype(int)
portfolio_11['CASA_DAY_SINCE_LAST_TXN'] = portfolio_11['CASA_DAY_SINCE_LAST_TXN'].astype(int)

In [ ]:
portfolio_10

In [ ]:
portfolio_10['REACTIVATED'].value_counts(dropna=False)

In [ ]:
portfolio_11['REACTIVATED'].value_counts(dropna=False)

In [ ]:
portfolio_10 = portfolio_10[portfolio_10['CUST_STATUS'].isin(['NGU DONG', 'DONG BANG'])]
portfolio_10.shape

In [ ]:
portfolio_11 = portfolio_11[portfolio_11['CUST_STATUS'].isin(['NGU DONG', 'DONG BANG'])]
portfolio_11.shape

# PORTFOLIO 10

In [ ]:
# d1 include d2
d1 = portfolio_10.sort_values(by='SCORE', ascending=False)
d1

In [ ]:
d1['REACTIVATED'].value_counts()

## AUC

In [ ]:
from sklearn import metrics
tmp = d1[['REACTIVATED', 'SCORE']].dropna()
auc = metrics.roc_auc_score(tmp['REACTIVATED'], tmp['SCORE'])
gini = 2 * auc - 1
auc, gini

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(5, 5))
metrics.RocCurveDisplay.from_predictions(tmp['REACTIVATED'], tmp['SCORE'], plot_chance_level=True, ax=axs)
axs.set_title('ROC-AUC Backtest')
plt.grid = True
plt.show()

## BINNING

In [ ]:
num_buckets = 10

In [ ]:
d1['BUCKET'], bins = pd.qcut(d1['SCORE'], num_buckets, labels=range(10, 0, -1), retbins=True)
d1['BUCKET'] = d1['BUCKET'].astype(int)

In [ ]:
d1['BUCKET'].unique()

In [ ]:
d1['BUCKET'].value_counts().sort_index()

In [ ]:
bins

### CAP

In [ ]:
tmp = d1[['REACTIVATED', 'BUCKET']].dropna()
tmp['REACTIVATED'] = tmp['REACTIVATED'].astype(int)
tmp = tmp.groupby('BUCKET')['REACTIVATED'].value_counts().to_frame('num_cust').reset_index().pivot(index='BUCKET', columns='REACTIVATED', values='num_cust')
tmp['TOTAL_LEAD'] = tmp[0] + tmp[1]
tmp

In [ ]:
total_reactivated = tmp[1].sum()
total_leads = tmp['TOTAL_LEAD'].sum()
total_leads, total_reactivated

In [ ]:
tmp['perc_REACTIVATED'] = tmp[1] / total_reactivated * 100
tmp['cumsum_REACTIVATED'] = tmp[1].cumsum()
tmp['perc_cumsum_REACTIVATED'] = tmp['cumsum_REACTIVATED'] / total_reactivated * 100
tmp['lift'] = tmp['perc_cumsum_REACTIVATED'] / (tmp['TOTAL_LEAD'].cumsum() / total_leads)  / 100

In [ ]:
tmp

## PORTFOLIO CHARACTERISTIC - OCT

In [ ]:
# let 
d11 = d1[d1['REACTIVATED']==1] # reactive only
d12 = d1[(d1['BUCKET']==1) & (d1['REACTIVATED']==0)] # bucket 1 - no-reactived

d11['REACTIVATED'] = d11['REACTIVATED'].astype(int)
d12['REACTIVATED'] = d12['REACTIVATED'].astype(int)

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(10,10))
d11['AREA_NAME'].value_counts().sort_values(ascending=False).plot(kind='pie', autopct='%.1f', colormap='Set2', ax=axs[0])
d12['AREA_NAME'].value_counts().sort_values(ascending=False).plot(kind='pie', autopct='%.1f', colormap='Set2', ax=axs[1])
# d2[d2['BUCKET']==1]['AREA_NAME'].value_counts().sort_values(ascending=False).plot(kind='pie', autopct='%.1f', colormap='Set2', ax=axs[2])
plt.show()

In [ ]:
tmp = d11
tmp1 = (tmp.reset_index().groupby([pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='3M', sort=True)])['CUSTOMER_CDE'].nunique() / len(tmp)).to_frame('prop').reset_index()
tmp1 = tmp.groupby(pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='6M', sort=True))['CUSTOMER_CDE'].nunique().reset_index()
fig, axs = plt.subplots(2,1, figsize=(20, 10))
sns.barplot(tmp1, x='CUST_OPEN_DT',y='CUSTOMER_CDE', ax=axs[0])

tmp = d12
tmp1 = (tmp.reset_index().groupby([pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='3M', sort=True)])['CUSTOMER_CDE'].nunique() / len(tmp)).to_frame('prop').reset_index()
tmp1 = tmp.groupby(pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='6M', sort=True))['CUSTOMER_CDE'].nunique().reset_index()
sns.barplot(tmp1, x='CUST_OPEN_DT',y='CUSTOMER_CDE', ax=axs[1])

axs[0].tick_params(axis='x', rotation=45)
axs[1].tick_params(axis='x', rotation=45)
plt.show()

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15,5))
sns.kdeplot(d11, x='SCORE', hue='REACTIVATED', fill=True, common_norm=False, ax=axs[0])
sns.kdeplot(d12, x='SCORE', hue='BUCKET', fill=True, common_norm=False, ax=axs[1])
# sns.kdeplot(d2, x='SCORE', hue='BUCKET', fill=True, common_norm=False, alpha=0.3, linewidth=0.3, palette='crest', ax=axs[2])
plt.show()

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15,5))
sns.histplot(d11, x='TOTAL_BAL', hue='REACTIVATED', log_scale=True, stat='density', common_norm=False,ax=axs[0])
sns.histplot(d12, x='TOTAL_BAL', hue='BUCKET', log_scale=True, stat='density', common_norm=False,ax=axs[1])
# sns.histplot(d2, x='TOTAL_BAL', hue='BUCKET', log_scale=True, stat='density', common_norm=False,palette='crest',ax=axs[2])
plt.show()

In [ ]:
d11['TOTAL_LOAN'].sum(), d12['TOTAL_LOAN'].sum()

In [ ]:
# f, axs = plt.subplots(1,2,figsize=(15,5))
# tmp = d1[d1['REACTIVATED']==1]
# tmp = tmp[tmp['TOTAL_LOAN']>0]
# print(len(tmp))
# # sns.kdeplot(tmp, x='TOTAL_LOAN', hue='REACTIVATED', log_scale=False, ax=axs[0])

# tmp = d1[(d1['BUCKET']==1) & (d1['REACTIVATED']==0)]
# tmp = tmp[tmp['TOTAL_LOAN']>0]
# print(len(tmp))
# # sns.kdeplot(tmp, x='TOTAL_LOAN', hue='BUCKET', log_scale=False, ax=axs[1])
# # sns.histplot(d2, x='TOTAL_LOAN', hue='BUCKET', log_scale=True, stat='density',palette='crest', alpha=0.3, ax=axs[2])
# # axs[0].set_xlim([0, 1e10])
# # axs[1].set_xlim([0, 1e10])
# plt.show()

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15,5))
sns.histplot(d11, x='MONTH_TYPE', hue='REACTIVATED', fill=True, stat='count',common_norm=True, linewidth=0.3,ax=axs[0])
sns.histplot(d12, x='MONTH_TYPE', hue='BUCKET', fill=True, stat='count',common_norm=True, linewidth=0.3,ax=axs[1])
# sns.histplot(d2, x='MONTH_TYPE', hue='BUCKET', fill=True, stat='count',common_norm=True, alpha=0.3, linewidth=0.3,palette='crest', ax=axs[2])
plt.show()

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15,5))
tmp = d11
tmp = tmp[tmp['MONTH_TYPE']<37]
sns.histplot(tmp,x='MONTH_TYPE', hue='REACTIVATED',common_norm=False,ax=axs[0])
# tmp = d1[(d1['BUCKET']>7) & (d1['MONTH_TYPE']<37) & (d1['MONTH_TYPE']>1)].copy()
tmp = d12
tmp = tmp[tmp['MONTH_TYPE']<37]
sns.histplot(tmp,x='MONTH_TYPE', hue='BUCKET',common_norm=False, ax=axs[1])
# sns.kdeplot(d2[(d2['MONTH_TYPE']<37) & (d2['MONTH_TYPE']>1) & (d2['BUCKET']>7)],x='MONTH_TYPE', hue='BUCKET',common_norm=False, palette='tab10',ax=axs[2])
plt.show()

# MAPPING BUCKET - NOV

In [ ]:
d2['CUST_STATUS'].value_counts()

In [ ]:
d2['REACTIVATED'].value_counts()

In [ ]:
# exclude reactivated in port-10
d2 = portfolio_11[~portfolio_11['CUSTOMER_CDE'].isin(d11['CUSTOMER_CDE'])].sort_values(by='SCORE', ascending=False)
d2

In [ ]:
criteria = [
    d2['SCORE'].between(bins[i], bins[i+1], inclusive='right') for i in range(num_buckets)
]
values=range(10, 0, -1)

In [ ]:
d2['BUCKET'] = np.select(criteria, values, 0)

In [ ]:
d2['BUCKET'].value_counts().sort_index()

## BUCKET MAPPING

In [ ]:
tmp = d2[['REACTIVATED', 'BUCKET']].dropna()
tmp['REACTIVATED'] = tmp['REACTIVATED'].astype(int)
tmp = tmp.groupby('BUCKET')['REACTIVATED'].value_counts().to_frame('num_cust').reset_index().pivot(index='BUCKET', columns='REACTIVATED', values='num_cust')
tmp['TOTAL_LEAD'] = tmp[0] + tmp[1]
tmp

In [ ]:
total_reactivated = tmp[1].sum()
total_leads = tmp['TOTAL_LEAD'].sum()
total_leads, total_reactivated

In [ ]:
tmp['perc_REACTIVATED'] = tmp[1] / total_reactivated * 100
tmp['cumsum_REACTIVATED'] = tmp[1].cumsum()
tmp['perc_cumsum_REACTIVATED'] = tmp['cumsum_REACTIVATED'] / total_reactivated * 100
tmp['lift'] = tmp['perc_cumsum_REACTIVATED'] / (tmp['TOTAL_LEAD'].cumsum() / total_leads)  / 100

In [ ]:
tmp

## PORTFOLIO-MAPPING CHARACTERISTIC

In [ ]:
d2.groupby('BUCKET')[['TOTAL_LOAN', 'TOTAL_BAL']].describe()

In [ ]:
d21 = d2[d2['BUCKET']==1]

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(10,15))
d11['AREA_NAME'].value_counts().sort_values(ascending=False).plot(kind='pie', autopct='%.1f', colormap='Set2', ax=axs[0])
d12['AREA_NAME'].value_counts().sort_values(ascending=False).plot(kind='pie', autopct='%.1f', colormap='Set2', ax=axs[1])
d21['AREA_NAME'].value_counts().sort_values(ascending=False).plot(kind='pie', autopct='%.1f', colormap='Set2', ax=axs[2])
plt.show()

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(20, 15))

tmp = d11
tmp1 = (tmp.reset_index().groupby([pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='3M', sort=True)])['CUSTOMER_CDE'].nunique() / len(tmp)).to_frame('prop').reset_index()
tmp1 = tmp.groupby(pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='6M', sort=True))['CUSTOMER_CDE'].nunique().reset_index()

sns.barplot(tmp1, x='CUST_OPEN_DT',y='CUSTOMER_CDE', ax=axs[0])

tmp = d12
tmp1 = (tmp.reset_index().groupby([pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='3M', sort=True)])['CUSTOMER_CDE'].nunique() / len(tmp)).to_frame('prop').reset_index()
tmp1 = tmp.groupby(pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='6M', sort=True))['CUSTOMER_CDE'].nunique().reset_index()
sns.barplot(tmp1, x='CUST_OPEN_DT',y='CUSTOMER_CDE', ax=axs[1])

tmp = d21
tmp1 = (tmp.reset_index().groupby([pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='3M', sort=True)])['CUSTOMER_CDE'].nunique() / len(tmp)).to_frame('prop').reset_index()
tmp1 = tmp.groupby(pd.Grouper(key='CUST_OPEN_DT', axis=0,freq='6M', sort=True))['CUSTOMER_CDE'].nunique().reset_index()
sns.barplot(tmp1, x='CUST_OPEN_DT',y='CUSTOMER_CDE', ax=axs[2])

axs[0].tick_params(axis='x', rotation=45)
axs[1].tick_params(axis='x', rotation=45)
axs[2].tick_params(axis='x', rotation=45)
plt.show()

In [ ]:
f, axs = plt.subplots(1,3,figsize=(15,5))
sns.kdeplot(d11, x='SCORE', hue='REACTIVATED', fill=True, common_norm=False, alpha=0.3, linewidth=0.3, palette='crest', ax=axs[0])
sns.kdeplot(d12, x='SCORE', hue='BUCKET', fill=True, common_norm=False, alpha=0.3, linewidth=0.3, palette='crest', ax=axs[1])
sns.kdeplot(d21, x='SCORE', hue='BUCKET', fill=True, common_norm=False, alpha=0.3, linewidth=0.3, palette='crest', ax=axs[2])
plt.show()

In [ ]:
f, axs = plt.subplots(1,3,figsize=(15,5))
sns.histplot(d11, x='TOTAL_BAL', hue='REACTIVATED', log_scale=True, stat='density', common_norm=False,ax=axs[0])
sns.histplot(d12, x='TOTAL_BAL', hue='BUCKET', log_scale=True, stat='density', common_norm=False,palette='crest',ax=axs[1])
sns.histplot(d21, x='TOTAL_BAL', hue='BUCKET', log_scale=True, stat='density', common_norm=False,palette='crest',ax=axs[2])
plt.show()

In [ ]:
f, axs = plt.subplots(1,3,figsize=(15,5))
sns.histplot(d11, x='TOTAL_LOAN', hue='REACTIVATED', log_scale=True, stat='density', palette='crest', alpha=0.3, ax=axs[0])
sns.histplot(d12, x='TOTAL_LOAN', hue='BUCKET', log_scale=True, stat='density', palette='crest', alpha=0.3, ax=axs[1])
sns.histplot(d21, x='TOTAL_LOAN', hue='BUCKET', log_scale=True, stat='density',palette='crest', alpha=0.3, ax=axs[2])
# axs[0].set_xlim([0, 1e10])
# axs[1].set_xlim([0, 1e10])
plt.show()

In [ ]:
f, axs = plt.subplots(1,3,figsize=(15,5))
sns.histplot(d11, x='MONTH_TYPE', hue='REACTIVATED', fill=True, stat='count',common_norm=True, linewidth=0.3,ax=axs[0])
sns.histplot(d12, x='MONTH_TYPE', hue='BUCKET', fill=True, stat='count',common_norm=True, alpha=0.3, linewidth=0.3,palette='crest', ax=axs[1])
sns.histplot(d21, x='MONTH_TYPE', hue='BUCKET', fill=True, stat='count',common_norm=True, alpha=0.3, linewidth=0.3,palette='crest', ax=axs[2])
plt.show()


In [ ]:
f, axs = plt.subplots(1,3,figsize=(15,5))
# d1['MONTH_TYPE'] = d1['MONTH_TYPE'].astype(int)
# d2['MONTH_TYPE'] = d2['MONTH_TYPE'].astype(int)
tmp = d11[d11['MONTH_TYPE']<37]
sns.histplot(tmp,x='MONTH_TYPE', hue='REACTIVATED',common_norm=False,ax=axs[0])

tmp = d12[d12['MONTH_TYPE']<37]
sns.histplot(tmp,x='MONTH_TYPE', hue='BUCKET',common_norm=False, palette='tab10',ax=axs[1])

tmp = d21[(d21['MONTH_TYPE']<37) & (d21['MONTH_TYPE']>1)]
sns.histplot(tmp,x='MONTH_TYPE', hue='BUCKET',common_norm=False, palette='tab10',ax=axs[2])
plt.show()